In [4]:
import rasterio as rio
import numpy as np
import libpysal as ps 
import pandas as pd
from PIL import Image

In [88]:
chirps_t = "../data/raw/segmented/10S60W/chirps_2001_10S60W.tif"
chirps_p = "../data/raw/segmented/10S60W/chirps-v2.0.2001_segmented.tif"
chirps = "../data/raw/chirps/chirps-v2.0.2001.tif"
chirps_s = "../data/raw/segmented/chirps_2001.tif"

In [99]:
with rio.open(chirps_t) as src:
    datat = src.read()
    datat = np.squeeze(data).astype("float32")
    
with rio.open(chirps_p) as src:
    datap = src.read()
    datap = np.squeeze(data).astype("float32")
with rio.open(chirps_s) as src:
    datas = src.read()
    datas = np.squeeze(data).astype("float32")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [82]:
LAT = -10
LONG = -60
with rio.open(chirps) as src:
    data = src.read()
    data = np.squeeze(data).astype("float32")

lat_index = np.arange(0, data.shape[0])
long_index = np.arange(0, data.shape[1])

A = src.transform
ymin, xmin = rio.transform.rowcol(A, -60, -20)
ymax, xmax = rio.transform.rowcol(A, -50, -10)

seg_data = data[ymax:ymin, xmin:xmax]
chirps_2001 = Image.fromarray(seg_data)
chirps_2001.save("../data/raw/segmented/chirps_2001.tif")


(2000, 7200)

array([[2224.9453, 2213.5957, 2227.5608, ..., 1784.4402, 1723.7512,
        1712.4493],
       [2227.0645, 2256.6042, 2246.2063, ..., 1778.2191, 1771.9319,
        1734.282 ],
       [2230.7144, 2266.6091, 2308.1462, ..., 1774.7363, 1759.7183,
        1731.4569],
       ...,
       [ 861.0691,  897.737 ,  911.3354, ..., 1134.9917, 1134.5109,
        1121.02  ],
       [ 877.6898,  896.6368,  921.1529, ..., 1127.404 , 1124.0513,
        1110.252 ],
       [ 870.9298,  900.1557,  907.9926, ..., 1121.4924, 1122.8226,
        1118.6578]], dtype=float32)

array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
      dtype=float32)

array([], shape=(0, 200), dtype=float32)

In [52]:
target_coords = (-10, -60)
length = 10
lat_max = target_coords[0]
lat_min = lat_max - length
long_min = target_coords[1]
long_max = long_min + length

# length variables
xlen = data.shape[0]
ylen = data.shape[1]

xmin, xmax = None, None
for x in range(xlen):
    if long[x] > long_min:
        xmin = x if xmin == None else xmin
    if long[xlen - x - 1] < long_max:
        xmax = xlen - x - 1 if xmax == None else xmax

# find ymin and ymax
ymin, ymax = None, None
for y in range(ylen):
    if lat[ylen - y - 1] > lat_min:
        ymin = ylen - y - 1 if ymin == None else ymin
    if lat[y] < lat_max:
        ymax = y if ymax == None else ymax

IndexError: list index out of range

In [10]:
from osgeo import gdal
ds = gdal.Open(chirps)
width = ds.RasterXSize
height = ds.RasterYSize
gt = ds.GetGeoTransform()
minx = gt[0]
miny = gt[3] + width*gt[4] + height*gt[5] 
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3] 

KeyboardInterrupt: 

In [32]:
def load_data(path):
    with rio.open(value) as src:
        data = src.read()
        data = np.squeeze(data).astype("float32")
    return data, src
def get_lat_long(data, src):
    # index array
    lat_index = np.arange(0, data.shape[0])
    long_index = np.arange(0, data.shape[1])
    
    # getting long and lat
    A = src.transform
    lat, long = rio.transform.xy(A, lat_index, long_index)
    
    # reshaping to shape of original data
    lat = np.array(lat)
    long = np.array(long)
    
    return lat, long

def segment_coords(data, lat, long, target_coords = (0, 0), border = 0, length = 10):
    # latitude/longitude variables
    lat_max = target_coords[0]
    lat_min = lat_max - length
    long_min = target_coords[1]
    long_max = long_min + length
    
    # length variables
    xlen = data.shape[1]
    ylen = data.shape[0]
    
    # find xmin and xmax
    xmin, xmax = None, None
    for x in range(xlen):
        if long[0, x] > long_min:
            xmin = x if xmin == None else xmin
        if long[0, xlen - x - 1] < long_max:
            xmax = xlen - x - 1 if xmax == None else xmax
            
    # find ymin and ymax
    ymin, ymax = None, None
    for y in range(ylen):
        if lat[ylen - y - 1, 0] > lat_min:
            ymin = ylen - y - 1 if ymin == None else ymin
        if lat[y, 0] < lat_max:
            ymax = y if ymax == None else ymax
            
    # index data from top-bottom, left-right
    return data[ymax:ymin + border, xmin:xmax + border]

In [ ]:
def load_data(path):
    with rio.open(value) as src:
        data = src.read()
        data = np.squeeze(data).astype("float32")
    return data, src
def get_lat_long(data, src):
    # index array
    lat_index = np.arange(0, data.shape[0])
    long_index = np.arange(0, data.shape[1])
    
    # getting long and lat
    A = src.transform
    lat, long = rio.transform.xy(A, lat_index, long_index)
    
    # reshaping to shape of original data
    lat = np.array(lat)
    long = np.array(long)
    
    return lat, long

def segment_coords(data, lat, long, target_coords = (0, 0), border = 0, length = 10):
    # latitude/longitude variables
    lat_max = target_coords[0]
    lat_min = lat_max - length
    long_min = target_coords[1]
    long_max = long_min + length
    
    # length variables
    xlen = data.shape[1]
    ylen = data.shape[0]
    
    # find xmin and xmax
    xmin, xmax = None, None
    for x in range(xlen):
        if long[0, x] > long_min:
            xmin = x if xmin == None else xmin
        if long[0, xlen - x - 1] < long_max:
            xmax = xlen - x - 1 if xmax == None else xmax
            
    # find ymin and ymax
    ymin, ymax = None, None
    for y in range(ylen):
        if lat[ylen - y - 1, 0] > lat_min:
            ymin = ylen - y - 1 if ymin == None else ymin
        if lat[y, 0] < lat_max:
            ymax = y if ymax == None else ymax
            
    # index data from top-bottom, left-right
    return data[ymax:ymin + border, xmin:xmax + border